In [73]:
import pandas as pd
import json

In [74]:
DATA = "data.txt"

In [75]:
with open(DATA, "r") as f:
    d = f.readlines()

In [76]:
d.pop(0)
d.pop(0)

'\n'

In [77]:
for e, x in enumerate(d[1::2]):
    json.loads(json.loads(x.replace("\\x", "_UNICX_")))

In [78]:
e*2+4, x


(1366,
 '"{\\"p2p_snapshot\\":[{\\"address\\":\\"185.15.75.179\\",\\"client_ver\\":\\"0.16.2\\",\\"conn_id\\":null,\\"degree\\":10,\\"key_id\\":\\"90b9231bf32560929fad338c88d0f91497e8db85490737eef3d0ddd38f1b66af70438e04fe58d74ded21a663721ca02755ebbc297317c31c16038882f9a0f5be\\",\\"listen_port\\":40102,\\"node_name\\":\\"DK15\\",\\"port\\":50888,\\"verified\\":true},{\\"address\\":\\"84.10.50.51\\",\\"client_ver\\":\\"0.16.2\\",\\"conn_id\\":null,\\"degree\\":10,\\"key_id\\":\\"258ab76a7ca7e49fb1ea6febba4c389b13d76cb0c8ef4a889a462ed857ac9ba972047f645680b725decd8266b39687ba2380702ea0eb175bde44f18b65073c78\\",\\"listen_port\\":40102,\\"node_name\\":\\"tiger-zoo\\",\\"port\\":59838,\\"verified\\":true},{\\"address\\":\\"46.139.47.15\\",\\"client_ver\\":\\"0.16.2\\",\\"conn_id\\":\\"dc8cbc65-b067-4c8c-b32d-81adaba6e816\\",\\"degree\\":10,\\"key_id\\":\\"7c4e8a897ad6ce1e19bfca4f2a553ddccbdb802a95269d8bcf06338ba5c3be1a89274a6c5d725903696858d7732c47c92a7e252634d665c6b4522072e53e38b2\\",\\"list

In [80]:
keys = [k.replace("p2pstats.", "")[:-1] for k in d[::2]]

vals = [json.loads(json.loads(v.replace("\\x", "_UNICX_"))) for v in d[1::2]]

assert len(keys) == len(vals)

network_all = {k: len(vv["p2p_snapshot"]) for k, vv in zip(keys, vals)}

In [ ]:
network_all["f052d3e4404c5f07842f1104b677d79569e81b39458eab516090947d29bf3e3cdd3847808a2e353d7be1717f9d4a4bce15079a42db6c843641ca52f8d2114dad"]

In [ ]:
# list of 
# [(k[:5], v) for k, v in sorted(list(network_all.items()), key=lambda x: x[1], reverse=True)]

In [ ]:
network = {k: [v["key_id"] for v in vv["p2p_snapshot"]] for k, vv in zip(keys, vals)}

In [ ]:
len(network)

In [ ]:
keys[0]

In [ ]:
# Format
# {
#   "nodes":[
# 		{"name":"node1","group":1},
# 		{"name":"node2","group":2},
# 		{"name":"node3","group":2},
# 		{"name":"node4","group":3}
# 	],
# 	"links":[
# 		{"source":2,"target":1,"weight":1},
# 		{"source":0,"target":2,"weight":3}
# 	]
# }

In [ ]:
num_node = {k: e for e, k in enumerate(keys)}

In [ ]:
connections_set = set((k1, k2) for k1 in keys for k2 in network[k1])

In [ ]:
# Assymetric connections

t = []

for k1, k2 in connections_set:
    if not (k2, k1) in connections_set:
        if k2 in num_node:
            t.append((k1, k2))
len(t)

In [ ]:
# Dead nodes?

r = []

for k1, k2 in connections_set:
    if k2 not in num_node:
        r.append(k2)
len(r)

In [ ]:
ports = [[(v["port"], v["listen_port"]) for v in vv["p2p_snapshot"]] for vv in vals]

In [ ]:
# Strange port behaviour - one node can have incoming cnnections on multiple ports

ps = []

for p in ports:
    if any(p[0][1] != q[1] for q in p):
        ps.append(p)
len(ps)

In [ ]:
connections_set_sorted = set(tuple(sorted([k1, k2])) for k1 in keys for k2 in network[k1])

In [ ]:
out = {}
out["nodes"] = [{"name": "", "group": 1} for k in keys]

In [ ]:
out["links"] = [{"source": num_node[k1], "target": num_node[k2], "weight": 1} 
                for k1, k2 in connections_set_sorted
                if k1 in num_node and k2 in num_node]

In [ ]:
len(out["links"])

In [ ]:
len(out["nodes"])

In [ ]:
with open("graphFile.json", "w") as f:
    json.dump(out, f)

In [ ]:
[(k, v) for k,v  in network.items() if k[:5] == "3752"]

In [ ]:
[(k, v) for k,v  in network.items() if any(vv[:5] == "3752" for vv in v)]

In [ ]:
# Format 2
# {
#   "nodes":[
# 		{"id":"node1","group":1},
# 		{"id":"node2","group":2},
# 	],
# 	"links":[
# 		{"source":"node1","target":"node2","value":1},
# 	]
# }

In [ ]:
out2 = {}

def get_degree(k):
    return network_all[k]

out2["nodes"] = [{"id": k, "group": get_degree(k)} for k in keys]

In [ ]:
out2["links"] = [{"source": k1, "target": k2, "value": 1} 
                for k1, k2 in connections_set
                if k1 in keys and k2 in keys]

In [ ]:
with open("graphFile2.json", "w") as f:
    json.dump(out2, f)